In [18]:
import pandas as pd
import os
from datetime import timedelta

## Filtraggio dei Dati Meteorologici

Questo script Python carica un file CSV contenente dati meteorologici, filtra le righe in base a un intervallo di date specificato e salva i dati filtrati in un nuovo file CSV.

### Dettagli del Codice:

1. **Caricamento del CSV**: Il file CSV contenente i dati meteorologici viene caricato in un DataFrame.
2. **Conversione della Data**: La colonna `ts_get` viene convertita in formato datetime per consentire il filtraggio.
3. **Filtraggio**: Vengono selezionate solo le righe comprese tra il 1° agosto 2024 e il 1° ottobre 2024.
4. **Salvataggio**: I dati filtrati vengono salvati in un nuovo file CSV.


In [5]:
file_path = "SmartCityQCWizard\weather\weather-010824-061024.csv"

df = pd.read_csv(file_path)

df['ts_get'] = pd.to_datetime(df['ts_get'])

start_date = '2024-08-01'
end_date = '2024-10-1'
filtered_df = df[(df['ts_get'] >= start_date) & (df['ts_get'] <= end_date)]

output_file = "weather-010824-300924.csv"
filtered_df.to_csv(output_file, index=False)

print(f"File salvato come: {output_file}")


File salvato come: weather-010824-300924.csv


## Unione di File CSV di Presenza

Questo script Python carica più file CSV contenenti dati di presenza, li unisce in un unico DataFrame e salva il risultato in un nuovo file CSV.

### Dettagli del Codice:

1. **Definizione dei Percorsi**: Viene creato un elenco di percorsi ai file CSV di presenza.
2. **Caricamento dei CSV**: Ogni file CSV viene caricato in un DataFrame utilizzando una lista di comprensione.
3. **Unione dei DataFrame**: I DataFrame caricati vengono concatenati in un unico DataFrame.
4. **Salvataggio**: Il DataFrame combinato viene salvato in un nuovo file CSV.


In [8]:
file_list = ["SmartCityQCWizard\\unique_attendance_15\\presenza_15_010824-140824.csv", 
             "SmartCityQCWizard\\unique_attendance_15\\presenza_15_010924-140924.csv", 
             "SmartCityQCWizard\\unique_attendance_15\\presenza_15_011024-081024.csv", 
             "SmartCityQCWizard\\unique_attendance_15\\presenza_15_150824-310824.csv", 
             "SmartCityQCWizard\\unique_attendance_15\\presenza_15_150924_300924.csv"]

df_list = [pd.read_csv(file) for file in file_list]
combined_df = pd.concat(df_list)

output_file = "presenza_010824-300924.csv"
combined_df.to_csv(output_file, index=False)

print(f"File combinato salvato come: {output_file}")

File combinato salvato come: presenza_010824-300924.csv


## Elaborazione e Suddivisione dei Dati di Traffico e Meteo

Questo script Python carica dati di traffico e meteorologici, espande i dati meteorologici per allinearsi meglio agli intervalli di tempo del traffico, e divide i dati in file CSV separati per ciascuna area di analisi.

### Dettagli del Codice:

1. **Caricamento dei Dati**: I file CSV contenenti i dati di traffico e meteo vengono caricati in DataFrame.
2. **Filtraggio delle Aree**: I dati di traffico vengono suddivisi in base all'area di analisi, creando un DataFrame separato per ciascuna area di Cagliari.
3. **Espansione dei Dati Meteorologici**: I dati meteorologici vengono espansi per includere timestamp a intervalli di 15 minuti attorno all'ora originale.
4. **Rounding e Merge**: I timestamp del traffico e dei dati meteorologici vengono arrotondati al quarto d'ora più vicino e uniti in un unico DataFrame.
5. **Pulizia dei Dati**: Le colonne duplicate e non necessarie vengono rimosse, e le colonne vengono riorganizzate.
6. **Divisione e Salvataggio**: I dati vengono divisi in file CSV per ciascuna area, oltre a salvare un file completo contenente tutti i dati uniti.


In [9]:
df = pd.read_csv(r"presenza_010824-300924.csv")

df0 =df[df['areaAnalisi'].str.contains('Intero Comune')]
df1 =df[df['areaAnalisi'].str.contains('Cagliari - 001')]
df2 =df[df['areaAnalisi'].str.contains('Cagliari - 002')]
df3 =df[df['areaAnalisi'].str.contains('Cagliari - 003')]
df4 =df[df['areaAnalisi'].str.contains('Cagliari - 004')]
df5 =df[df['areaAnalisi'].str.contains('Cagliari - 005')]
df6 =df[df['areaAnalisi'].str.contains('Cagliari - 006')]
df7 =df[df['areaAnalisi'].str.contains('Cagliari - 007')]
df8 =df[df['areaAnalisi'].str.contains('Cagliari - 008')]
df9 =df[df['areaAnalisi'].str.contains('Cagliari - 009')]
df10 =df[df['areaAnalisi'].str.contains('Cagliari - 010')]
df11 =df[df['areaAnalisi'].str.contains('Cagliari - 011')]
df12 =df[df['areaAnalisi'].str.contains('Cagliari - 012')]

In [17]:

output_dir = 'dataset_output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_traffico = pd.read_csv(r"presenza_010824-300924.csv")
df_meteo = pd.read_csv(r"weather-010824-300924.csv")

df_traffico['datetime'] = pd.to_datetime(df_traffico['datetime'])
df_meteo['ts_get'] = pd.to_datetime(df_meteo['ts_get'])

# Funzione per espandere i dati meteo mantenendo ts_get originale
def expand_weather_data(df_meteo):
    expanded_data = []
    
    for _, row in df_meteo.iterrows():
        base_time = row['ts_get']
        original_ts = row['ts_get']  # Manteniamo il timestamp originale
        
        # Genera timestamp per ±30 minuti intorno all'ora del dato meteo
        for minutes in [-15, 0, 15, 30]:
            new_time = base_time + timedelta(minutes=minutes)
            new_row = row.copy()
            new_row['matching_time'] = new_time  
            new_row['ts_get'] = original_ts    
            expanded_data.append(new_row)
    
    return pd.DataFrame(expanded_data)

df_meteo_expanded = expand_weather_data(df_meteo)

# Arrotonda i timestamp al quarto d'ora più vicino per il matching
df_traffico['datetime_rounded'] = df_traffico['datetime'].dt.round('15min')
df_meteo_expanded['datetime_rounded'] = df_meteo_expanded['matching_time'].dt.round('15min')

# Esegui il merge dei dataset
df_unito = pd.merge(df_traffico,
                    df_meteo_expanded,
                    left_on='datetime_rounded',
                    right_on='datetime_rounded',
                    how='left')

# Pulizia delle colonne duplicate e non necessarie
df_unito = df_unito.drop(['datetime_rounded', 'matching_time'], axis=1)

# Riorganizza le colonne per avere le date all'inizio
cols = df_unito.columns.tolist()
cols = ['datetime', 'ts_get'] + [col for col in cols if col not in ['datetime', 'ts_get']]
df_unito = df_unito[cols]

# Dizionario delle regioni
regioni = {
    'Cagliari - 001': 'df1',
    'Cagliari - 002': 'df2',
    'Cagliari - 003': 'df3',
    'Cagliari - 004': 'df4',
    'Cagliari - 005': 'df5',
    'Cagliari - 006': 'df6',
    'Cagliari - 007': 'df7',
    'Cagliari - 008': 'df8',
    'Cagliari - 009': 'df9',
    'Cagliari - 010': 'df10',
    'Cagliari - 011': 'df11',
    'Cagliari - 012': 'df12'
}

# Divisione e salvataggio dei dataframe per area
for area, nome_df in regioni.items():
    df_area = df_unito[df_unito['areaAnalisi'].str.contains(area)]
    output_file = os.path.join(output_dir, f"{nome_df}.csv")
    df_area.to_csv(output_file, index=False, date_format='%Y-%m-%d %H:%M:%S')
    print(f"File salvato: {output_file}")

# Salva anche il dataset completo
output_file_completo = os.path.join(output_dir, "dataset_completo.csv")
df_unito.to_csv(output_file_completo, index=False, date_format='%Y-%m-%d %H:%M:%S')
print(f"File completo salvato: {output_file_completo}")

File salvato: dataset_output\df1.csv
File salvato: dataset_output\df2.csv
File salvato: dataset_output\df3.csv
File salvato: dataset_output\df4.csv
File salvato: dataset_output\df5.csv
File salvato: dataset_output\df6.csv
File salvato: dataset_output\df7.csv
File salvato: dataset_output\df8.csv
File salvato: dataset_output\df9.csv
File salvato: dataset_output\df10.csv
File salvato: dataset_output\df11.csv
File salvato: dataset_output\df12.csv
File completo salvato: dataset_output\dataset_completo.csv
